# Protocol timer

This is the protocol timer base. It uses a `.csv` source file to dynamically build a protocol. The steps are then sent to a self-hosted Matrix-based chatroom (closed source alternative: Discord) using an HTTP webhook. The protocol can be sequentially automatically by using the `Excecute cell and below` button on VSCode. If you need time to prepare between steps, you can use the `Execute Cell` button to start the timer for only a single step, advancing one step at a time.

## Initialization

### Load Libraries

In [12]:
from IPython.core.getipython import get_ipython
from IPython.display import display, Markdown, Latex
import pandas as pd
from time import time, sleep
from tqdm.notebook import tqdm
from tqdm import trange
import datetime as date
import requests
import json
import os
import markdown
import re

# from tqdm.contrib.telegram import tqdm, trange
from configparser import ConfigParser


# from timer_functions import *

In [13]:
if os.path.exists("token.secrets"):
	parser = ConfigParser()
	_ = parser.read('token.secrets')
	channel_id = parser.get('discord', 'channel_id')
	webhook_url = parser.get('discord', 'webhook')
	matrix_webhook_url = parser.get('matrix', 'webhook')

else:
	channel_id = os.getenv("CHANNEL_ID")
	webhook_url = os.getenv("DISCORD_WEBHOOK")
	matrix_webhook_url = os.getenv("MATRIX_WEBHOOK")

In [14]:
# Adjust for timezone

# America/Chicago == -6; -5 during DST
timezone_offset = -5.0

In [15]:
# Define functions used in this notebook
def calculate_seconds(timestring):
    time_obj = date.datetime.strptime(timestring, "%H:%M:%S")
    hours = time_obj.hour
    minutes = time_obj.minute
    seconds = time_obj.second

    # Calculate the total number of seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    return total_seconds


def step_commands(df, index_num):
    line1 = f"print('**{df.Combined.iloc[index_num]}**')"
    total_seconds = calculate_seconds(df.Duration.iloc[index_num])

    line2 = f"print('Duration is {round(total_seconds/60, 3)} minutes')"

    line3 = f"countdown(df.Duration.iloc[{index_num}], df.Combined.iloc[{index_num}], df.Duration.iloc[{index_num}])"

    combined_lines = line1 + " \n" + line2 + " \n" + line3

    return combined_lines

def get_protocol_description(protocol_csv_path):
    file_name = os.path.basename(protocol_csv_path)
    file_name_without_extension = os.path.splitext(file_name)[0]

    # Creating the description variable with "md" extension
    description_path = os.path.join(os.path.dirname(protocol_csv_path), f'{file_name_without_extension}.md')

    # Read the Markdown content from the file
    with open(description_path, 'r', encoding='utf-8') as file:
        markdown_content = file.read()

    # Convert Markdown to HTML
    html_content = markdown.markdown(markdown_content)

    # Compress HTML into a single-line string
    html_description = re.sub('\s+', ' ', html_content).strip()

    return html_description



def countdown(timestring, step_text, duration):

    # Display / send messages
    display(f"Starting {step_text}") # to console
    # send_discord_message(f"Starting {step_text}.") # Discord
    send_matrix_protocol_step(step_text, duration) # Matrix




    time_obj = date.datetime.strptime(timestring, "%H:%M:%S")

    hours = time_obj.hour
    minutes = time_obj.minute
    seconds = time_obj.second

    # Calculate the total number of seconds
    total_seconds = hours * 3600 + minutes * 60 + seconds

    # While loop that checks if total_seconds reaches zero
    # If not zero, decrement total time by one second
    total_seconds_original = total_seconds

    pbar = tqdm(
        # total=100,
        total=total_seconds,
        bar_format="{desc}   {percentage:3.0f}% [Elapsed: {elapsed} -- Remaining: {remaining}]",
    )
    while total_seconds > 0:
        # # Timer represents time left on countdown
        timer = date.timedelta(seconds=total_seconds)

        # print(timer, sep='\r')

        # Update the tqdm progress bar
        # pbar.update(100 / total_seconds_original)

        sleep(1)

        total_seconds -= 1
        pbar.update(1)

    pbar.refresh()


    # Display / Send completed messages
    display(f"COMPLETED {step_text}")

    # send_discord_message(f"COMPLETED {step_text}.") # Discord


def compose_discord_json(message):
    json_payload = {
        "content": message,
        "username": "Protocol Timer",
        "avatar_url": "https://raw.githubusercontent.com/github/explore/149e057770c384ddaba0393b369c2c8f16e433bb/topics/jupyter-notebook/jupyter-notebook.png",
        "tts": True,
        "embeds": [
            {
                "title": "Protocol Timer",
                "type": "rich",
                "timestamp": date.datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%SZ"),
                "fields": [
                    {
                        "name": "Protocol Step",
                        "value": message,
                    },
                ],
                "footer": {
                    "text": "Protocol-Timer created by Pranav Mishra (pranavmishra90/protocol-timer)"
                },
            }
        ],
    }

    return json_payload

#### Matrix ####

def compose_matrix_protocol_json(message, duration):
    duration_parts = duration.split(":")
    duration_var = date.timedelta(hours=int(duration_parts[0]), minutes=int(duration_parts[1]), seconds=int(duration_parts[2]))

    tzinfo = date.timezone(date.timedelta(hours=timezone_offset))

    current_datetime = date.datetime.now(tzinfo)
    resulting_datetime = current_datetime + duration_var

    step_end_time = resulting_datetime.strftime("%I:%M %p")

    json_payload = {
        "html": f"<h2>{message}</h2><b>Duration</b>: {duration}</br><b>End Time</b>: {step_end_time}"
    }
    return json_payload



def send_matrix_protocol_step(message, duration):
    json_payload = compose_matrix_protocol_json(message, duration)
    headers = {"Content-Type": "application/json"}

    response = requests.post(
        matrix_webhook_url, data=json.dumps(json_payload), headers=headers
    )

    if response.status_code == 202:
        print("Matrix notification successful")
    else:
        print(f"Failed to notify on Matrix. Status code: {response.status_code}")


def compose_matrix_message_json(message):
    json_payload = {
        "html": f"{message}"
    }
    return json_payload

def send_matrix_message(message):
    json_payload = compose_matrix_message_json(message)
    headers = {"Content-Type": "application/json"}

    response = requests.post(
        matrix_webhook_url, data=json.dumps(json_payload), headers=headers
    )

    if response.status_code == 202:
        print("Matrix notification successful")
    else:
        print(f"Failed to notify on Matrix. Status code: {response.status_code}")


#### Discord ####

def send_discord_message(message):
    json_payload = compose_discord_json(message)

    headers = {"Content-Type": "application/json"}

    response = requests.post(
        webhook_url, data=json.dumps(json_payload), headers=headers
    )

    if response.status_code == 204:
        print("Discord notification successful!")
    else:
        print(f"Failed to notify on discord. Status code: {response.status_code}")


def current_datetime():
    current_datetime = date.datetime.now()

    # Format the datetime as "Day, mm/dd/yy at hh:mm:ss AM/PM"
    formatted_datetime = current_datetime.strftime("%A, %m/%d/%y at %I:%M:%S %p")

    return formatted_datetime


### Create a new cell in VSCode ###
def create_new_cell(contents):
    shell = get_ipython()

    payload = dict(
        source="set_next_input",
        text=contents,
        replace=False,
    )
    shell.payload_manager.write_payload(payload, single=False)


clear_chat_page = "<i>(Clearing the page)</i>" + "<br>" * 70

## Import the protocol and format the dataframe

In [16]:
protocol_csv_path = './histology/Deparaffinization.csv' #'./histology/FFPE.csv' 


# Show the HTML Description file loaded
html_description = get_protocol_description(protocol_csv_path)
print(html_description)

<h1>Deparaffinization Protocol</h1> <p><strong>Pranav Kumar Mishra</strong><br /> Department of Surgery and Orthopedic Surgery<br /> Laboratories of Alfonso Torquati and Anna Spagnoli<br /> Rush University, Chicago, IL, USA </p> <h2>Overview</h2> <p>This is a protocol for preparing slides using a formalin fixed, paraffin embedded (FFPE) section. These slides will be used for various protocols, including:</p> <ul> <li>Hematoxylin and Eosin</li> <li>Immunohistochemistry</li> <li>Spatial Transcriptomics</li> </ul> <p>The protocol was originally written by Michael Kluppel and has been modified with the following changes for the BFGI project</p> <h3>Changes</h3> <p>(none)</p>


In [17]:
df = pd.read_csv(protocol_csv_path)

#flip the dataframe since a new "step" is always inserted immediately after the currently running one (new cells end up in reverse order)
df = df.iloc[::-1].reset_index()
df.drop(columns='index', inplace=True)

df['Combined'] = "Step "+ df['Step Number'].astype(str) + ": " + df['Step Text'].astype(str) + " (" + df['Duration'].astype(str) + ")"

df = df.astype({'Step Number': 'int', 'Step Text': 'string', 'Duration': 'string', 'Description': 'string', 'Comment': 'string', 'Combined':'string'})

display(df)

,Step Number,Step Text,Duration,Description,Comment,Combined
0,16,PBS - 3,0:05:00,Room Temperature,<NA>,Step 16: PBS - 3 (0:05:00)
1,15,PBS - 2,0:05:00,Room Temperature,<NA>,Step 15: PBS - 2 (0:05:00)
2,14,PBS - 1,0:05:00,Room Temperature,<NA>,Step 14: PBS - 1 (0:05:00)
3,13,0.1% Triton X-100 in PBS,0:20:00,Room Temperature,Tissue permeabilization,Step 13: 0.1% Triton X-100 in PBS (0:20:00)
4,12,PBS - 3,0:05:00,Room Temperature,<NA>,Step 12: PBS - 3 (0:05:00)
5,11,PBS - 2,0:05:00,Room Temperature,<NA>,Step 11: PBS - 2 (0:05:00)
6,10,PBS - 1,0:05:00,Room Temperature,<NA>,Step 10: PBS - 1 (0:05:00)
7,9,50% Ethanol,0:05:00,Room Temperature,<NA>,Step 9: 50% Ethanol (0:05:00)
8,8,75% Ethanol,0:05:00,Room Temperature,<NA>,Step 8: 75% Ethanol (0:05:00)
9,7,95% Ethanol,0:05:00,Room Temperature,<NA>,Step 7: 95% Ethanol (0:05:00)


In [18]:
# Calculate the total time and expected time of completion for the protocol

# Create a new column with a timedelta format
df['Timedelta'] = pd.to_timedelta(df['Duration'])

total_duration = df['Timedelta'].sum()
total_hours, remainder = divmod(total_duration.total_seconds(), 3600)
total_minutes, total_seconds = divmod(remainder, 60)

total_time_formatted = f"{int(total_hours)}h:{int(total_minutes)}m:{int(total_seconds)}s"

# Calculate the expected time of completion
expected_time_completion = date.datetime.now() + total_duration
formatted_completion_time = expected_time_completion.strftime('%A at %I:%M %p')

display(Markdown(f"Total time to complete the protocol: {total_time_formatted}.<br>Expected time of completion: {formatted_completion_time}"))

Total time to complete the protocol: 2h:30m:0s.<br>Expected time of completion: Friday at 06:58 PM

## Your Protocol
**Instructions**
A cell has been created for each step of your protocol according to the durations specified. When you "run" a cell, a progress bar will appear, with a timer for the elapsed time and a time remaining countdown.

After the timer runs out, a completed message will be displayed.

### Run the next cell *once*

In [19]:
create_new_cell("#------ End of protocol ------# \n #When you have reached this point, you can delete all of the protocol steps which have been created, or simply close this file without saving")

create_new_cell('send_matrix_message(f"---- PROTOCOL COMPLETE at {current_datetime()} ---- <br><hr>")')

for row in df.index:
    all_in_one = step_commands(df, row)
    
    create_new_cell(all_in_one)

display(Markdown('## Start the protocol\n**When you are ready to begin, run the cell below**'))

create_new_cell('send_matrix_message(f"---- STARTING PROTOCOL at {current_datetime()} ---- <br>Total time to complete the protocol: {total_time_formatted}.<br>Expected time of completion: {formatted_completion_time}")')

create_new_cell('send_matrix_message(f"{html_description}")')

create_new_cell('send_matrix_message(clear_chat_page)')

create_new_cell("#------ Click here, then run all of the cells below ------#")

## Start the protocol
**When you are ready to begin, run the cell below**

In [20]:
#------ Click here, then run all of the cells below ------#

In [21]:
send_matrix_message(clear_chat_page)

Matrix notification successful


In [22]:
send_matrix_message(f"{html_description}")

Matrix notification successful


In [23]:
send_matrix_message(f"---- STARTING PROTOCOL at {current_datetime()} ---- <br>Total time to complete the protocol: {total_time_formatted}.<br>Expected time of completion: {formatted_completion_time}")

Matrix notification successful


In [24]:
print('**Step 0: Protocol created by Michael Kluppel (0:00:00)**') 
print('Duration is 0.0 minutes') 
countdown(df.Duration.iloc[16], df.Combined.iloc[16], df.Duration.iloc[16])

**Step 0: Protocol created by Michael Kluppel (0:00:00)**
Duration is 0.0 minutes


'Starting Step 0: Protocol created by Michael Kluppel (0:00:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 0: Protocol created by Michael Kluppel (0:00:00)'

In [ ]:
print('**Step 1: Vacuum oven (1:00:00)**') 
print('Duration is 60.0 minutes') 
countdown(df.Duration.iloc[15], df.Combined.iloc[15], df.Duration.iloc[15])

In [25]:
print('**Step 2: Xylene (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[14], df.Combined.iloc[14], df.Duration.iloc[14])

**Step 2: Xylene (0:05:00)**
Duration is 5.0 minutes


'Starting Step 2: Xylene (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 2: Xylene (0:05:00)'

In [26]:
print('**Step 3: Xylene (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[13], df.Combined.iloc[13], df.Duration.iloc[13])

**Step 3: Xylene (0:05:00)**
Duration is 5.0 minutes


'Starting Step 3: Xylene (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 3: Xylene (0:05:00)'

In [27]:
print('**Step 4: 50% Xylene / 50% Ethanol (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[12], df.Combined.iloc[12], df.Duration.iloc[12])

**Step 4: 50% Xylene / 50% Ethanol (0:05:00)**
Duration is 5.0 minutes


'Starting Step 4: 50% Xylene / 50% Ethanol (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 4: 50% Xylene / 50% Ethanol (0:05:00)'

In [28]:
print('**Step 5: 100% Ethanol - 1 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[11], df.Combined.iloc[11], df.Duration.iloc[11])

**Step 5: 100% Ethanol - 1 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 5: 100% Ethanol - 1 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 5: 100% Ethanol - 1 (0:05:00)'

In [29]:
print('**Step 6: 100% Ethanol - 2 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[10], df.Combined.iloc[10], df.Duration.iloc[10])

**Step 6: 100% Ethanol - 2 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 6: 100% Ethanol - 2 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 6: 100% Ethanol - 2 (0:05:00)'

In [30]:
print('**Step 7: 95% Ethanol (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[9], df.Combined.iloc[9], df.Duration.iloc[9])

**Step 7: 95% Ethanol (0:05:00)**
Duration is 5.0 minutes


'Starting Step 7: 95% Ethanol (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 7: 95% Ethanol (0:05:00)'

In [31]:
print('**Step 8: 75% Ethanol (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[8], df.Combined.iloc[8], df.Duration.iloc[8])

**Step 8: 75% Ethanol (0:05:00)**
Duration is 5.0 minutes


'Starting Step 8: 75% Ethanol (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 8: 75% Ethanol (0:05:00)'

In [32]:
print('**Step 9: 50% Ethanol (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[7], df.Combined.iloc[7], df.Duration.iloc[7])

**Step 9: 50% Ethanol (0:05:00)**
Duration is 5.0 minutes


'Starting Step 9: 50% Ethanol (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 9: 50% Ethanol (0:05:00)'

In [33]:
print('**Step 10: PBS - 1 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[6], df.Combined.iloc[6], df.Duration.iloc[6])

**Step 10: PBS - 1 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 10: PBS - 1 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 10: PBS - 1 (0:05:00)'

In [34]:
print('**Step 11: PBS - 2 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[5], df.Combined.iloc[5], df.Duration.iloc[5])

**Step 11: PBS - 2 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 11: PBS - 2 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 11: PBS - 2 (0:05:00)'

In [35]:
print('**Step 12: PBS - 3 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[4], df.Combined.iloc[4], df.Duration.iloc[4])

**Step 12: PBS - 3 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 12: PBS - 3 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 12: PBS - 3 (0:05:00)'

In [36]:
print('**Step 13: 0.1% Triton X-100 in PBS (0:20:00)**') 
print('Duration is 20.0 minutes') 
countdown(df.Duration.iloc[3], df.Combined.iloc[3], df.Duration.iloc[3])

**Step 13: 0.1% Triton X-100 in PBS (0:20:00)**
Duration is 20.0 minutes


'Starting Step 13: 0.1% Triton X-100 in PBS (0:20:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 13: 0.1% Triton X-100 in PBS (0:20:00)'

In [37]:
print('**Step 14: PBS - 1 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[2], df.Combined.iloc[2], df.Duration.iloc[2])

**Step 14: PBS - 1 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 14: PBS - 1 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 14: PBS - 1 (0:05:00)'

In [38]:
print('**Step 15: PBS - 2 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[1], df.Combined.iloc[1], df.Duration.iloc[1])

**Step 15: PBS - 2 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 15: PBS - 2 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 15: PBS - 2 (0:05:00)'

In [39]:
print('**Step 16: PBS - 3 (0:05:00)**') 
print('Duration is 5.0 minutes') 
countdown(df.Duration.iloc[0], df.Combined.iloc[0], df.Duration.iloc[0])

**Step 16: PBS - 3 (0:05:00)**
Duration is 5.0 minutes


'Starting Step 16: PBS - 3 (0:05:00)'

Matrix notification successful


     0% [Elapsed: 00:00 -- Remaining: ?]

'COMPLETED Step 16: PBS - 3 (0:05:00)'

In [40]:
send_matrix_message(f"---- PROTOCOL COMPLETE at {current_datetime()} ---- <br><hr>")

Matrix notification successful


In [41]:
#------ End of protocol ------# 
 #When you have reached this point, you can delete all of the protocol steps which have been created, or simply close this file without saving